In [147]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('../src')) # or the path to your source code
sys.path.insert(0, module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [148]:
import pandas as pd
import json
import re
from time import sleep

In [149]:
# Instancia para Irriapp
from kilimo_data_lib.irriapp import build_irriapp_client
irriapp_client = build_irriapp_client()

In [172]:
# Instancia para Planet
from kilimo_data_lib.planet import build_planet_client
planet = build_planet_client()

In [151]:
# Import geolib
from kilimo_data_lib.geolib import add_buffer_to_geojson

In [156]:
# Leo el dataframe
df = pd.read_csv('dataset_agro_cultiv_reg.csv')

In [157]:
df.head(3)

,Tipo de manejo del campo/lote,Link a control del campo,Id de los lotes que tienen el mismo manejo,Fecha desde,Detalles,Success,field_id,plot_id
0,1,https://control.kilimo.com.ar/field/17529/cont...,"34980, 34979, 33539, 33538, 33537, 33536, 33535",1/1/2015,No pudieron precisar la fecha de inicio del us...,Pia,17529.0,34980
1,1,https://control.kilimo.com.ar/field/17529/cont...,"34980, 34979, 33539, 33538, 33537, 33536, 33535",1/1/2015,No pudieron precisar la fecha de inicio del us...,Pia,17529.0,33537
2,1,https://control.kilimo.com.ar/field/17529/cont...,"34980, 34979, 33539, 33538, 33537, 33536, 33535",1/1/2015,No pudieron precisar la fecha de inicio del us...,Pia,17529.0,33538


In [158]:
# Mani plots out of dataset
df = df[~df['plot_id'].isin([631, 694, 632])]

In [162]:
df.shape[0]

134

In [ ]:
# Consulto el product_id de SWC en la suborganización
planet_my_products = planet.get_my_products()
planet_my_products

In [160]:
# ID del producto SWC de Planet
planet_product_id = planet_my_products['results'][0]['id']

In [161]:
planet_product_id

475

In [163]:
# Crear una collection en Planet para el proyecto
#title = '20241111_humedad_suelo_dataset_success'
#description = 'Analysis of increase soil water retention in agronomy success plots'
title = 'Soil_Water_Holding_Capacity'
description = 'Detecting increases of holding capacity'
planet_collection = planet.create_collection(title, description)

In [164]:
planet_collection_id = planet_collection['id']

In [165]:
planet_collection_id

'soil_water_holding_capacity-prol0RZ'

In [106]:
path_prefix = "test_template_subs/CR"

In [185]:
start_time = "2018-01-01T00:00:00Z"  # Start date for all planet subscriptions
path_prefix = "Soil_Water_Holding_Capacity/SWHC"  # Prefix for S3 delivery

plot_id_processed_order = []
resp = []
planet_create_subscription_result = []
planet_reservation_result = []

# Recorro el df y obtengo el plot_id de cada lote a suscribir
for index, row in df.iterrows():

    if index <= 56:
        continue
    
    plot_id = row['plot_id']
    plot_id_processed_order.append(plot_id)

    # Get plot data
    plot = irriapp_client.get_plot(plot_id)
    
    # Get geojson from irriapp
    plot_geojson = irriapp_client.get_plot_geojson(plot_id)

    string_plot_geojson = json.dumps(plot_geojson)

    # Add 1m buffer to geojson
    output_name = f'{plot_id}_geojson_with_buffer'
    plot_geojson_buffer = add_buffer_to_geojson(output_name, string_plot_geojson)

    # Add geojson to collection
    title = f'{plot['pk']}-{plot['name']}-{plot['field_name']}'
    description = f'descripcion_{plot['pk']}'
    
    resp.append(planet.add_features_to_collection(planet_collection_id, 
                                                  plot_geojson_buffer,
                                                  str(plot_id),
                                                  title, description))
    
    reference = resp[-1]
    # Reserve quota for collection
    planet_reservation_result.append(planet.reserve_quota_bulk(planet_product_id, reference))

    sleep(3)

    # Create suscription for plot
    planet_create_subscription_result.append(planet.create_subscription_by_reference(plot_id, reference[0], 
                                                                                     start_time, path_prefix=path_prefix))

    print('Index:', index, 'plot_id:', plot_id)

Index: 57 plot_id: 14557
Index: 58 plot_id: 14270
Index: 59 plot_id: 14272
Index: 60 plot_id: 14271
Index: 61 plot_id: 31007
Index: 62 plot_id: 31001
Index: 63 plot_id: 31000
Index: 64 plot_id: 30996
Index: 65 plot_id: 30995
Index: 66 plot_id: 30997
Index: 67 plot_id: 31004
Index: 68 plot_id: 31002
Index: 69 plot_id: 33329
Index: 70 plot_id: 33326
Index: 71 plot_id: 33328
Index: 72 plot_id: 33327
Index: 73 plot_id: 33330
Index: 74 plot_id: 33339
Index: 75 plot_id: 33340
Index: 76 plot_id: 33336
Index: 77 plot_id: 33337
Index: 78 plot_id: 33338
Index: 79 plot_id: 35025
Index: 80 plot_id: 35024
Index: 81 plot_id: 35093
Index: 82 plot_id: 35028
Index: 83 plot_id: 35026
Index: 84 plot_id: 35027
Index: 85 plot_id: 27289
Index: 86 plot_id: 27299
Index: 87 plot_id: 27292
Index: 88 plot_id: 27297
Index: 89 plot_id: 27287
Index: 90 plot_id: 27281
Index: 91 plot_id: 27302
Index: 92 plot_id: 27290
Index: 93 plot_id: 30439
Index: 94 plot_id: 30451
Index: 95 plot_id: 30447
Index: 96 plot_id: 30446


In [144]:
resp

[['pl:features/my/test_template_2-Lvxo9LZ/33537-g3aRaWm']]

In [189]:
with open('planet_reservation_result_It2.pkl', 'wb') as archivo:
    pickle.dump(planet_reservation_result, archivo)